In [ ]:
import time
import pandas as pd
import pandapower as pp
import numpy as np
import matplotlib.pyplot as plt
import os
from pandapower.plotting.plotly import pf_res_plotly
from pandapower.plotting.plotly import simple_plotly

import Utils.FeederBalancing as FeederBalancing
import Utils.GA as GA
import Utils.utils as utils

# Import Network

In [ ]:
input_path = '../Data/Input'
output_path = '../Data/Output'

feederbalancing = FeederBalancing.FeederBalancing(input_path)

feeder_colors = ['#ff7f0e', '#9467bd']
feeder_colors_after = [utils.scale_lightness(c, 0.6) for c in feeder_colors]

meaningful_days = [15, 83, 162, 241, 324]
timesteps = feederbalancing.get_meaningful_days_timesteps(meaningful_days)

# Visualize Time-Series

In [ ]:
feederbalancing.plot_P()

In [ ]:
clips = ([-12,15], [-0.2,7.5]) #To be changed depending on your data
a = utils.plot_P_by_feeder(feederbalancing.B_init, feederbalancing, timesteps, feeder_colors, meaningful_days=meaningful_days, clips=clips)

# Initialize GA Algorithm

In [ ]:
#Set weigths for the different terms in the objective function
feederbalancing.scale_unbalance = 80 / (feederbalancing.number_timesteps * feederbalancing.number_customers) #weight for unbalance between the phases (Eq. 7 in the paper)
feederbalancing.scale_aggregate = 0 / (feederbalancing.number_timesteps * feederbalancing.number_customers) #weight to avoid worsening the situation at any step (not mentioned in the paper but may lead to a more robust solution)
feederbalancing.scale_changes = 4 / feederbalancing.number_customers #weight to reduce the number of changes (Eq 10)
feederbalancing.scale_distances = 1 / feederbalancing.number_customers #weight for considering the distance of the reconfiguration (Eq 11)

In [ ]:
ga_instance = GA.GA(feederbalancing)

In [ ]:
ga_instance.reconstruct = False
ga_instance.feeder = 0
ga_instance.initial_solution = feederbalancing.B_init_nobinary
# ga_instance.initial_solution = solution

if(ga_instance.feeder == 0):
    ga_instance.num_generations = 50
    ga_instance.population_size = 30
else:
    ga_instance.num_generations = 6
    ga_instance.population_size = 10
ga_instance.mutation_rate = 0.4
# ga_instance.initialize_run()

In [ ]:
initial_loss = feederbalancing.objective_function(feederbalancing.B_init, False)
print(f'Initial loss: {initial_loss}. \n Initial config: {feederbalancing.B_init_nobinary}')

In [ ]:
ga_instance.runGA()
if(ga_instance.reconstruct):
    solution = ga_instance.reconstruct_solution(ga_instance.best_solution[0])
else:
    solution = ga_instance.best_solution[0]

In [ ]:
#Some plots to see the different losses
plt.plot(np.array(feederbalancing.unbalance_loss) * feederbalancing.scale_unbalance, label='Unbalance')
plt.plot(np.array(feederbalancing.associated_loss) * feederbalancing.scale_aggregate, label='Associated_loss')
plt.plot(np.array(feederbalancing.changes_loss) * feederbalancing.scale_changes, label='Changes')
plt.plot(np.array(feederbalancing.loss_distance) * feederbalancing.scale_distances, label='Distance')
plt.legend()

In [ ]:
if(ga_instance.reconstruct):
    solution = ga_instance.reconstruct_solution(ga_instance.best_solution[0])
else:
    solution = ga_instance.best_solution[0]

In [ ]:
print(f'Initial loss: {initial_loss}.\nInitial config: {feederbalancing.B_init_nobinary}. Number customers: {len(feederbalancing.B_init_nobinary)}')
B_sol = feederbalancing.get_B_from_genetic(solution)
print(f'Solution loss: {feederbalancing.objective_function(B_sol, False)} ({feederbalancing.objective_function(B_sol)}). N. changes: {np.sum(B_sol * feederbalancing.B_init_opposite)}. \n Solution config: {solution}')
print([k for k in feederbalancing.B_init_nobinary])
print([k for k in solution])

In [ ]:
A_init, P = utils.plot_P_by_feeder(feederbalancing.B_init, feederbalancing, timesteps, feeder_colors, meaningful_days=meaningful_days, clips=clips)

In [ ]:
A_sol, P_sol = utils.plot_P_by_feeder(B_sol, feederbalancing, timesteps, feeder_colors, meaningful_days=meaningful_days, clips=clips)

In [ ]:
utils.plot_feeder_unbalance(feederbalancing, A_init, A_sol, feeder_colors, feeder_colors_after, meaningful_days=meaningful_days)

# Run PFs

In [ ]:
_, results = feederbalancing.run_simulations(feederbalancing.B_init, output_path)

In [ ]:
utils.plot_PF_results(feederbalancing, results)

In [ ]:
_, results_sol = feederbalancing.run_simulations(B_sol, output_path)

In [ ]:
utils.plot_PF_results(feederbalancing, results_sol)

In [ ]:
#Plot a graph to see the change positions
solution = feederbalancing.get_B_from_genetic(solution)
for i,changed in enumerate(np.sum(solution * feederbalancing.B_init_opposite, axis=1)):
    if(changed==1):
        c = feederbalancing.net.asymmetric_load.iloc[i] #it may give issues if the indexes are not the same as expected
        feederbalancing.net.bus.at[c['bus'], 'color'] = 'red'
simple_plotly(feederbalancing.net, bus_color=feederbalancing.net.bus['color'])